In [2]:
from langchain.chains import create_citation_fuzzy_match_chain
from langchain.chat_models import ChatOpenAI

In [3]:
question = "What did the author do during college?"
context = """
My name is Jason Liu, and I grew up in Toronto Canada but I was born in China.
I went to an arts highschool but in university I studied Computational Mathematics and physics. 
As part of coop I worked at many companies including Stitchfix, Facebook.
I also started the Data Science club at the University of Waterloo and I was the president of the club for 2 years.
"""

In [4]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
chain = create_citation_fuzzy_match_chain(llm)
result = chain.run(question=question, context=context)
print(result)

question='What did the author do during college?' answer=[FactWithEvidence(fact='The author studied Computational Mathematics and physics in university.', substring_quote=['in university I studied Computational Mathematics and physics']), FactWithEvidence(fact='The author started the Data Science club at the University of Waterloo and was the president of the club for 2 years.', substring_quote=['started the Data Science club at the University of Waterloo', 'president of the club for 2 years'])]


In [6]:
def highlight(text, span):
    return (
        "..."
        + text[span[0] - 20 : span[0]]
        + "*"
        + "\033[91m"
        + text[span[0] : span[1]]
        + "\033[0m"
        + "*"
        + text[span[1] : span[1] + 20]
        + "..."
    )

In [7]:
for fact in result.answer:
    print("Statement:", fact.fact)
    for span in fact.get_spans(context):
        print("Citation:", highlight(context, span))
    print()

Statement: The author studied Computational Mathematics and physics in university.
Citation: ...arts highschool but *in university I studied Computational Mathematics and physics*. 
As part of coop I...

Statement: The author started the Data Science club at the University of Waterloo and was the president of the club for 2 years.
Citation: ...x, Facebook.
I also *started the Data Science club at the University of Waterloo* and I was the presi...
Citation: ...erloo and I was the *president of the club for 2 years*.
...



In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from langchain.chains.router import MultiRetrievalQAChain


# Chain for having a conversation based on retrieved documents.
from langchain.chains  import ConversationalRetrievalChain
import langchain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from pprint import pprint


info_about_berlin_loader = TextLoader("real-time-information-on-berlin.txt")
vectorstore = Chroma.from_documents(documents=info_about_berlin_loader.load(), 
                                    embedding=OpenAIEmbeddings())
real_time_info_on_berlin_retriever = vectorstore.as_retriever(search_kwargs={"k": 1})


my_musings_about_berlin = [
    """Once upon a time in Berlin, a mysterious key appeared in the hands of a humble street musician. 
    The key was said to unlock a secret door beneath the Brandenburg Gate, leading to a hidden world. 
    A talking bear from the Berlin Zoo guided the musician, revealing that he was the guardian of this magical realm. 
    Together, they navigated through enchanted U-Bahn tunnels filled with glittering gems and talking trains. 
    In the end, the musician's melody restored harmony to both worlds, and the bear returned to his post, forever grateful."""
]
my_musings_about_berlin_retriever = FAISS.from_texts(personal_texts, OpenAIEmbeddings()).as_retriever()



chatGPT = ChatOpenAI()
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

chat_with_memory_and_knowledge = ConversationalRetrievalChain.from_llm(
    llm=chatGPT,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 1}),
    memory=memory,
    # verbose=True
)

In [8]:
from langchain.chat_models import ChatOpenAI

In [9]:
from langchain.utilities import SQLDatabase
from langchain.chains import create_sql_query_chain

db = SQLDatabase.from_uri("sqlite:///../../../../../notebooks/Chinook.db")
query_chain = create_sql_query_chain(ChatOpenAI(temperature=0), db)

OperationalError: (sqlite3.OperationalError) unable to open database file
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [10]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.schema.document import Document
index_creator = VectorstoreIndexCreator()
index = index_creator.from_documents([Document(page_content="Foo")])
retriever = index.vectorstore.as_retriever()

In [11]:
from langchain.prompts import ChatPromptTemplate

system_message = """Use the information from the below two sources to answer any questions.

Source 1: a SQL database about employee data
<source1>
{source1}
</source1>

Source 2: a text database of random information
<source2>
{source2}
</source2>
"""

prompt = ChatPromptTemplate.from_messages([("system", system_message), ("human", "{question}")])

In [12]:
full_chain = {
    "source1": {"question": lambda x: x["question"]} | query_chain | db.run,
    "source2": (lambda x: x['question']) | retriever,
    "question": lambda x: x['question'],
} | prompt | ChatOpenAI()

NameError: name 'query_chain' is not defined

In [ ]:
response = full_chain.invoke({"question":"How many Employees are there"})
print(response)